The image of inital VHR is too big.
I need to cut that into smaller images.
Maybe I choose the size of smaller images as 2km * 2km

Input: a intial images; the size of each images.

Steps:
    1. a list of lons and lats with a internal of 2km (based on MODIS, 4 or 8 pixes)
    2. for the area within each lon and each lat, detect whether their are vhr images, and cut it if yes.


## Read raster with gdal

In [1]:
import gdal
one_vhr = "E:\\code\\Py_workplace\\satellite_flooding\\Data\\Germany\\Post\\20210718_10500500E6DD3C00.tif"

# Open .tif
# https://blog.csdn.net/xiaotiig/article/details/128297792
tif_data = gdal.Open(one_vhr)
# Size:
print(tif_data.RasterXSize)
print(tif_data.RasterYSize)
# geotransform: .GetGeoTransform(): 
# 0: The x-coordinate of the upper-left corner of the upper-left pixel (origin)
# 1: The pixel width in the x-direction (resolution)
# 2: The rotation angle in the x-direction (skew)
# 3: The y-coordinate of the upper-left corner of the upper-left pixel (origin)
# 4: The rotation angle in the y-direction (skew)
# 5: The pixel height in the y-direction (resolution, negative for north-up images)
geotransform = tif_data.GetGeoTransform()
print(geotransform)

# band
bandCount = tif_data.RasterCount
print(bandCount)

# projection
proj = tif_data.GetProjection()
print(proj)




(6.691455796986715, 6.201369844702031e-06, 0.0, 50.58876356793541, 0.0, -6.201369844702031e-06)
3
3
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


In [3]:
import gdal
import numpy as np
mask_size = 2
given_region = [6.691455797, 50.588763568, 7.084461410, 50.296027904]

''' for a given region, return a serious smaller masks to cover the region.
Input: given_region (xx), mask_size (km)
Output: a serious of [left, top, right, bottom]
'''
# initialization
masks_list = []
reso = 0.0001  # the resolution, about 10 meters
reso_meter = reso * 100000


# parameters for masks
xsize = mask_size * 1000 / reso_meter * reso
ysize = mask_size * 1000 / reso_meter * reso


longitudes = np.arange(given_region[0], given_region[2], xsize, dtype = float).tolist()
latitudes = np.arange(given_region[1], given_region[3], -ysize, dtype = float).tolist()



for lon in longitudes:
    for lat in latitudes:
        left = lon
        top = lat
        right = left + reso+xsize
        bottom = top - reso+xsize
        masks_list.append([left, top, right, bottom])